<a href="https://colab.research.google.com/github/amir-asari/Introduction_to_Huggingface/blob/main/3_FineTuningaPretrainedModel/3_Fine_Tuning_Models_with_Trainer_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers torch gradio accelerate evaluate torchinfo

# 1: Standard Pipeline

1. Dataset
2. Tokenizer
3. Model
4. Model Output processing

## 1: Dataset & 2: Tokenizer

In [ ]:
import datasets     as huggingface_datasets     # Easy readablity
import transformers as huggingface_transformers # Easy readablity
import torchinfo

In [ ]:
glue_sentence_similarity_dataset  = huggingface_datasets.load_dataset("glue", "mrpc")                           # 1. dataset

bert_checkpoint                   = "bert-base-cased"
bert_tokenizer                    = huggingface_transformers.AutoTokenizer.from_pretrained(bert_checkpoint)     # 2. tokenizer -> data to numeric_ids
def tokenize_function(example):
    return bert_tokenizer (example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets                = glue_sentence_similarity_dataset.map(tokenize_function, batched=True)       # 2. tokenized dataset
data_collator                     = huggingface_transformers.DataCollatorWithPadding(tokenizer= bert_tokenizer)


In [ ]:
len(tokenized_datasets['train'])

3668

## 3: Pre-trained Model Architecture

In [ ]:
# 3. Model
#             = huggingface_transformers. AutoTokenizer.                      from_pretrained (bert_checkpoint)
model         = huggingface_transformers. AutoModelForSequenceClassification. from_pretrained (bert_checkpoint, num_labels = 2, force_download= True)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


For Detailed Architecture - Study Following
1. [Epic History of Large Language Models (LLMs) - CampusX](https://www.youtube.com/watch?v=8fX3rOjTloc)
2. **Attention, GPT, BERT** - Fundamental Building Blocks of Entire NLP Field


### Important methods

1. **vars(VARIABLE) in python**
2. print(VARIABLE)
3. **model.state_dict()**
4. **torchinfo.summary()**

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# 2 Main Parts of model
model.bert            # 1. Pretrained Bert by Google
model.dropout         #
model.classifier      # 2. Task Specific HEAD for Intelligence

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
# PRE TRAINED INTELLIGENCE - in BERT by GOOGLE
# 3 Main Parts of BERT model
model.bert.embeddings # 1. Word -> Semantic Vector of Word Meaning
model.bert.encoder    # 2. Semantic Vector -> + Language Understanding & Enriching the semantic vector. Extracting all possible features
model.bert.pooler     # 3. Compress Feature map if too big for RAM

BertPooler(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (activation): Tanh()
)

In [ ]:
model.bert.encoder

BertEncoder(
  (layer): ModuleList(
    (0-11): 12 x BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (output): BertSelfOutput(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (intermediate): BertIntermediate(
        (dense): Linear(in_features=768, out_features=3072, bias=True)
        (intermediate_act_fn): GELUActivation()
      )
      (output): BertOutput(
        (dense): Linear(in_features=3072, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [ ]:
# HEAD PART OF THE INTELLIGENCE
model.classifier

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
vars(model.classifier)

{'training': False,
 '_parameters': OrderedDict([('weight',
               Parameter containing:
               tensor([[-0.0238,  0.0248,  0.0220,  ..., -0.0142, -0.0218,  0.0100],
                       [-0.0135, -0.0219,  0.0206,  ...,  0.0110, -0.0483, -0.0173]],
                      requires_grad=True)),
              ('bias',
               Parameter containing:
               tensor([0., 0.], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules'

In [ ]:
model.classifier.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0181, -0.0424,  0.0137,  ...,  0.0115,  0.0136, -0.0006],
                      [-0.0292, -0.0283, -0.0123,  ...,  0.0264, -0.0117,  0.0155]])),
             ('bias', tensor([ 4.0921e-05, -4.0921e-05]))])

In [ ]:
torchinfo.summary(model, col_names = ["num_params", "params_percent", "trainable"], depth = 4, row_settings = ["var_names"], verbose = 1);
# verbose = 2 is important. Understanding internal

Layer (type (var_name))                                                Param #                   Param %                   Trainable
BertForSequenceClassification (BertForSequenceClassification)          --                             --                   True
├─BertModel (bert)                                                     --                             --                   True
│    └─BertEmbeddings (embeddings)                                     --                             --                   True
│    │    └─Embedding (word_embeddings)                                22,268,928                 20.56%                   True
│    │    └─Embedding (position_embeddings)                            393,216                     0.36%                   True
│    │    └─Embedding (token_type_embeddings)                          1,536                       0.00%                   True
│    │    └─LayerNorm (LayerNorm)                                      1,536                       

In [ ]:
torchinfo.summary(model, col_names = ["num_params", "params_percent", "trainable"], depth = 5, row_settings = ["var_names"]);

In [ ]:
model.config

BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.38.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [ ]:
for layer_name, params in model.named_parameters():
    print(layer_name, params.shape)
    break

bert.embeddings.word_embeddings.weight torch.Size([28996, 768])


# 2: Fine Tuning Model with `Huggingface Trainer API`

### Dataloader not needed in Trainer API.
- Can use RAM memory optimized dataset directly
- What if Dataset size >>>> RAM??? Either Dataloader in Pytorch or **Arrow Dataset** optimized for Huge size Datasets by huggingface

We only need to use huggingface datasets & huggingface transformers.

TrainingArguments Class - Important Arguments
- output_dir
- evaluation_strategy - steps vs epochs
- learning_rate
- logging_dir, log_level, logging_strategy

In [ ]:
training_args = huggingface_transformers.TrainingArguments(output_dir = "./test-trainer/")

trainer = huggingface_transformers.Trainer(
    model,
    training_args,
    train_dataset=  tokenized_datasets["train"],      # MEMORY OPTIMIZED DATASET directly. Not Pytorch Dataloader
    eval_dataset=   tokenized_datasets["validation"], # MEMORY OPTIMIZED DATASET directly. Not Pytorch Dataloader
    data_collator=  data_collator,
    tokenizer=      bert_tokenizer,
)
# Training starts with following
# trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
import numpy as np
import evaluate

# LOGITS vs PROBABILITY
predictions_logits  = trainer.predict(tokenized_datasets["validation"])
predicted_class     = np.argmax(predictions_logits.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=predicted_class, references=predictions_logits.label_ids)

{'accuracy': 0.3161764705882353, 'f1': 0.0}

In [ ]:
def compute_metrics(eval_prediction_logits):
    metric                      = evaluate.load("glue", "mrpc")

    predictions_logits, labels  = eval_prediction_logits
    predicted_class             = np.argmax(predictions_logits, axis=-1)
    metric_value = metric.compute(predictions =predicted_class, references=labels)
    return metric_value

In [ ]:
training_args = huggingface_transformers.TrainingArguments(output_dir = "test-trainer", evaluation_strategy= "epoch")
model         = huggingface_transformers.AutoModelForSequenceClassification.from_pretrained(bert_checkpoint, num_labels=2)

trainer = huggingface_transformers.Trainer(
    model,
    training_args,
    train_dataset=    tokenized_datasets["train"],      # MEMORY OPTIMIZED DATASET directly. Not Pytorch Dataloader
    eval_dataset=     tokenized_datasets["validation"],
    data_collator=    data_collator,
    tokenizer=        bert_tokenizer,
    compute_metrics=  compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.448205,0.803922,0.871383
